# Генетичний алгоритм

## Імпорт бібліотек

In [48]:
import random
import itertools
import math
from IPython.display import clear_output

Етапи:
- **Initialization:** population size, search space
- **Fitness function & Selection**
- **Generate next population: Crossover & Mutation**
- **Termination**

## Initialization

Спочатку створимо допоміжну функцію для генерації допустимої множини хромосом, які можуть бути в індивіда. У нашому випадку - це всі літери англійського алфавіта в нижньому регістрі та пробіл.

In [50]:
def make_search_space():
    alphabet = [chr(x) for x in range(ord('a'), ord('z') + 1)]
    return alphabet + [' ']

Тепер створимо функцію генерації індивідів з рандомно обраних хромосом, та функцію для генерації популяції таких індивідів потрібного розміру.

In [51]:
def random_individual(genotype_length, search_space):
    return [random.choice(search_space) for i in range(genotype_length)]

def init_generation(generation_size, genotype_length, search_space):
    return [random_individual(genotype_length, search_space) for i in range(generation_size)]

In [70]:
search_space = make_search_space()
generation = init_generation(5, 4, search_space)
generation

[['a', 'k', 'n', 's'],
 ['o', 'w', 'c', 'y'],
 ['t', 'b', ' ', 'h'],
 ['i', 'l', 'k', 'r'],
 ['d', 'f', 'k', 'l']]

## Fitness function & Selection

Спочатку напишемо fitness функцію для оцінки індивіда, на скільки наближеним є його набір хромосом до цільвого набору. 

In [71]:
target = ['d', 'a', 'r', 'w', 'i', 'n']
individual = ['c', 'a', 'r', 'l', 'i', 'n']

def loss(individual, target):
    comparisons = [i[0] == i[1] for i in zip(individual, target)]
    return sum(comparisons) / len(target)

loss(individual, target)

0.6666666666666666

Тепер, коли ми можемо оцінити кожного індивіда, напишемо функцію природнього добору, яка з усієї популяції обирає найкращих індивідів, які потім (пізніше реалізуємо це) дадуть нащадків.

In [73]:
def select(population, proportion=0.5):
    fitnesses = [loss(individual, target) for individual in population]
    fitnesses, population = (list(t) for t in zip(*sorted(zip(fitnesses, population), reverse=True)))
    threshold = math.ceil(len(population)*proportion)
    best_scores = fitnesses[:threshold]
    fittest_individuals = population[:threshold]
    return fittest_individuals, best_scores

In [133]:
target = ['d', 'a', 'r', 'w', 'i', 'n']
population = init_generation(10, 6, search_space)
selection_coeff = 0.5
best_individuals, best_scores = select(population, selection_coeff)
best_individuals, best_scores

([['d', ' ', 'v', 's', 'q', 'n'],
  [' ', 'v', 'r', 'w', 'd', 't'],
  ['y', 'g', 'l', 'c', 'i', 'g'],
  ['w', 'h', 'r', 'u', 'p', 'b'],
  ['t', 'w', 'y', 't', ' ', 'r']],
 [0.3333333333333333,
  0.3333333333333333,
  0.16666666666666666,
  0.16666666666666666,
  0.0])

## Generate next population: Crossover & Mutation

Спочатку напишемо функцію мутації хромосом індивіда.

In [105]:
def mutate(child, mutation_ratio, search_space=search_space):
    count = math.ceil(len(child) * mutation_ratio)
    for i in range(count):
        child[random.randint(0, len(child)-1)] = random.choice(search_space)
    return child

In [106]:
individual = ['d', 'a', 'r', 'w', 'i', 'n']
mutate(individual, 0.25)

['s', 'x', 'r', 'w', 'i', 'n']

Тепер напишемо функцію, створення нового індивіда шляхом схрещення генома двох батьків.

In [109]:
def produce_child(daddy, mom, shift=0):
    child = daddy.copy()
    child[shift::2] = mom[shift::2]
    return child

In [111]:
daddy = ['d', 'a', 'r', 'w', 'i', 'n']
mom = ['f', 'o', 'w', 'l', 'e', 'r']
produce_child(daddy, mom)

['f', 'a', 'w', 'w', 'e', 'n']

Маючи функції породження нових індивідів та внесення випадкових мутацій в іх геном, тепер ми можемо створити функцію генерації новою популяції.

In [119]:
def new_generation(parents):
    children = [produce_child(*random.sample(parents, k=2)) for i in parents]
    children = [mutate(child, 0.25) for child in children]
    new_generation = (parents + children)
    return new_generation

In [138]:
print(f"Parents:\n{best_individuals}\n")
new_population = new_generation(best_individuals)
print(f"New population (parents & children):\n{new_population}")

Parents:
[['d', ' ', 'v', 's', 'q', 'n'], [' ', 'v', 'r', 'w', 'd', 't'], ['y', 'g', 'l', 'c', 'i', 'g'], ['w', 'h', 'r', 'u', 'p', 'b'], ['t', 'w', 'y', 't', ' ', 'r']]

New population (parents & children):
[['d', ' ', 'v', 's', 'q', 'n'], [' ', 'v', 'r', 'w', 'd', 't'], ['y', 'g', 'l', 'c', 'i', 'g'], ['w', 'h', 'r', 'u', 'p', 'b'], ['t', 'w', 'y', 't', ' ', 'r'], ['y', 'w', 'd', 't', 'i', 'b'], ['i', 'j', 'v', 'u', 'q', 'b'], ['y', 'v', 'l', 't', 'i', 't'], ['t', ' ', 'r', 'z', 'd', 'n'], ['t', 'a', 'y', 's', ' ', 'p']]


## Create model

Зберемо все в одну модель:

In [142]:
class GA:
    def __init__(self, target, population_size=1000, mutation_ratio=0.10, selection_coeff=0.5):
        self.target = list(target)
        self.mutation_ratio = mutation_ratio
        self.selection_coeff = selection_coeff
        self.genotype_length = len(target)
        self.population_size = population_size
        self.search_space = self.make_search_space()
        self.generation = self.init_generation(self.population_size,
                                               self.genotype_length,
                                               self.search_space)
        
        self.output_string = "".join(self.generation[0])
    
    def make_search_space(self):
        alphabet = [chr(x) for x in range(ord('a'), ord('z') + 1)]
        return alphabet + [' ']
    
    def init_generation(self, generation_size, genotype_length, search_space):
        return [self.random_individual(genotype_length, search_space) for i in range(generation_size)]
    
    def random_individual(self, genotype_length, search_space):
        return [random.choice(search_space) for i in range(genotype_length)]
    
    def loss(self, individual, target):
        comparisons = [i[0] == i[1] for i in zip(individual, target)]
        return sum(comparisons) / len(target)
    
    def select(self, population, proportion=0.5):
        fitnesses = [self.loss(individual, self.target) for individual in population]
        fitnesses, population = (list(t) for t in zip(*sorted(zip(fitnesses, population), reverse=True)))
        threshold = math.ceil(len(population)*proportion)
        best_scores = fitnesses[:threshold]
        fittest_individuals = population[:threshold]
        return fittest_individuals, best_scores
    
    def new_generation(self, parents):
        children = [self.produce_child(*random.sample(parents, k=2)) for i in parents]
        children = [self.mutate(child, self.mutation_ratio) for child in children]
        new_generation = (parents + children)[:self.population_size]
        return new_generation
    
    def mutate(self, child, mutation_ratio):
        count = math.ceil(len(child) * mutation_ratio)
        for i in range(count):
            child[random.randint(0, len(child)-1)] = random.choice(self.search_space)
        return child 
        
    def produce_child(self, daddy, mom, shift=0):
        child = daddy.copy()
        child[shift::2] = mom[shift::2]
        return child
    
    def train(self):
        best_score = .0
        count = 0
        while best_score < 1:
            fittest_individuals, best_scores = self.select(self.generation, self.selection_coeff)
            best_score = best_scores[0]
            self.output_string = "".join(fittest_individuals[0])
            clear_output()
            print(self.output_string)
            print(f"Iteration: {count}; Best score: {best_scores[0]}\n")
            self.generation = self.new_generation(fittest_individuals)
            count += 1

In [150]:
ga = GA("to be or not to be that is the question", 100, .010)

In [151]:
ga.train()

to be or not to be that is the question
Iteration: 237; Best score: 1.0

